In [3]:
import sqlite3
import pyimgur
from matplotlib import use
import yfinance as yf
import ta
import requests
import pandas as pd
import numpy as np
import datetime,time,re
import mplfinance as mpf
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
from pandas.core.frame import DataFrame

dict_from_ETF_list = pd.read_excel("/home/awinlab/Documents/yeh/LineBot/FiveLineAPP/ETF_list.xlsx")
ETF_list = dict_from_ETF_list['Number']

In [4]:
def five_line(data):   
    timetrend = list(range(1, data.shape[0]+1))
    data['timetrend'] = timetrend
    data = data[['timetrend','Close']]
    data = data.dropna()
    reg = LinearRegression()
    x = data['timetrend'].to_frame()
    y = data['Close'].to_frame()
    reg.fit(x,y)
    
    a = reg.intercept_ #截距
    beta = reg.coef_ #斜率
    longtrend = a + beta*x
    res = np.array(list(data['Close'])) - np.array(list(longtrend['timetrend']))
    std = np.std(res,ddof=1)
    fiveline = pd.DataFrame()
    fiveline['+2SD'] = longtrend['timetrend'] + (2*std)
    fiveline['+1SD'] = longtrend['timetrend'] + (1*std)
    fiveline['TL'] = longtrend['timetrend']
    fiveline['-1SD'] = longtrend['timetrend'] - (1*std)
    fiveline['-2SD'] = longtrend['timetrend'] - (2*std) 
    use_fiveline = pd.merge(data, fiveline[['+2SD','+1SD','TL','-1SD','-2SD']], left_index=True, right_index=True, how='left')
    pick_fiveline = use_fiveline[['Close','+2SD','+1SD','TL','-1SD','-2SD']]
    return pick_fiveline

def BBands(data): #布林通道，非樂活通道
    data_bb = data.copy()
    indicator_bb = ta.volatility.BollingerBands(close = data_bb['Close'])#, window = 20, window_dev = 2)
    data_bb['bbh'] = indicator_bb.bollinger_hband()
    data_bb['bbm'] = indicator_bb.bollinger_mavg()
    data_bb['bbl'] = indicator_bb.bollinger_lband()
    data_bb = data_bb.dropna()
    use_BBands = pd.merge(data, data_bb[['bbh','bbm','bbl']], left_index=True, right_index=True, how='left')
    pick_BBands = use_BBands[['bbh','bbm','bbl']]
    return pick_BBands

def KD(data):
    data_KD = data.copy()
    data_KD['min'] = data_KD['Low'].rolling(9).min()
    data_KD['max'] = data_KD['High'].rolling(9).max()
    data_KD['RSV'] = 100*(data_KD['Close'] - data_KD['min'])/(data_KD['max'] - data_KD['min'])
    data_KD = data_KD.dropna()

    K_list = [50]
    for num,rsv in enumerate(list(data_KD['RSV'])):
        K_yestarday = K_list[num]
        K_today = 2/3 * K_yestarday + 1/3 * rsv
        K_list.append(K_today)
    data_KD['K'] = K_list[1:]

    D_list = [50]
    for num,K in enumerate(list(data_KD['K'])):
        D_yestarday = D_list[num]
        D_today = 2/3 * D_yestarday + 1/3 * K
        D_list.append(D_today)
    data_KD['D'] = D_list[1:]
    use_KD = pd.merge(data, data_KD[['K','D']], left_index=True, right_index=True, how='left')
    pick_KD = use_KD[['K','D']]
    return pick_KD

In [3]:

today = datetime.date.today()
print(today)
end1 = today - datetime.timedelta(days=5)
print(end1)
end2 = today + datetime.timedelta(days=1)
print(end2)
df = yf.download(f"0050.TW", start = str(today), end = str(end2))
print(df)

2022-06-27
2022-06-22
2022-06-28
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-06-27  119.800003  121.449997  119.800003  121.099998  121.099998   

              Volume  
Date                  
2022-06-27  16536788  


In [5]:
def daily_update():
    today = datetime.date.today()
    end = today + datetime.timedelta(days=1)
    conn = sqlite3.connect('ETF_list.db')
    cursor = conn.cursor()
    for i in ETF_list:
        print(i)
        print(today)
        print(end)
        df = yf.download(f"{i}.TW", start = str(today), end = str(end))
        print(df)
        if df.empty:
            print('no data today')
            continue
        #=======================Strategy_data==================================
        f = five_line(df)
        b = BBands(df)
        k = KD(df)
        merge_df = pd.merge(b,k, left_index=True, right_index=True, how='left')
        merge2_df = pd.merge(f, merge_df, left_index=True, right_index=True, how='left')
        merge2_df = merge2_df.dropna()
        merge2_df['Number'] = f'{i}'
        merge2_df['Date'] = merge2_df.index.strftime("%Y-%m-%d")
        pick_df = merge2_df[['Date','Number','Close','+2SD','+1SD','TL','-1SD','-2SD','bbh','bbm','bbl','K','D']]
        #print(len(pick_df))
        for j in range(len(pick_df)):
            print(j)
            cursor.execute("insert into Strategy_data(Date, Number, Close, '+2SD', '+1SD', TL, '-1SD', '-2SD', bbh, bbm, bbl, K, D) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", tuple(pick_df.iloc[j]))
            conn.commit()
            
        #=======================ETF_data==================================
        #today = datetime.date.today()
        #end = today + datetime.timedelta(days=1)
        #df = yf.download(f"{i}.TW", start = str(today), end = str(today))
        df['Number'] = f'{i}'
        df['Date'] = df.index.strftime("%Y-%m-%d")
        pick_df = df[['Date', 'Number','Open', 'High', 'Low', 'Close', 'Volume']]

        #print(pick_df.index)
        #print(pick_df)
        #print('股票：',i, pick_df)
        pick_df['Volume'] = pick_df['Volume'].map('{}'.format)

        #print(tuple(pick_df.iloc[0]))
        #print(len(pick_df))
        for i in range(len(pick_df)):
            cursor.execute('insert into ETF_data(Date, Number, Open, High, Low, Close, Volume) VALUES (?, ?, ?, ?, ?, ?, ?)', tuple(pick_df.iloc[i]))
            conn.commit()
            
        

In [6]:
daily_update()

0050
2022-07-06
2022-07-07
[*********************100%***********************]  1 of 1 completed
                  Open        High    Low  Close  Adj Close    Volume
Date                                                                 
2022-07-05  112.550003  112.949997  109.0  111.0      111.0  17026450
0051
2022-07-06
2022-07-07


/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[*********************100%***********************]  1 of 1 completed
             Open       High        Low      Close  Adj Close  Volume
Date                                                                 
2022-07-05  50.25  51.200001  49.849998  50.349998  50.349998   57686
0052
2022-07-06
2022-07-07
[*********************100%***********************]  1 of 1 completed
                 Open       High   Low      Close  Adj Close   Volume
Date                                                                 
2022-07-05  91.099998  91.099998  87.5  89.849998  89.849998  1030579


/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWa

0053
2022-07-06
2022-07-07
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low  Close  Adj Close  Volume
Date                                                                 
2022-07-05  52.950001  52.950001  51.599998  52.25      52.25   18889
0054
2022-07-06
2022-07-07
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 0054.TW: No data found for this date range, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
no data today
0055
2022-07-06
2022-07-07


...

/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWa

[*********************100%***********************]  1 of 1 completed
                 Open   High        Low  Close  Adj Close  Volume
Date                                                             
2022-07-05  24.129999  24.15  24.129999  24.15      24.15   32000
00708L
2022-07-06
2022-07-07
[*********************100%***********************]  1 of 1 completed
                 Open       High    Low  Close  Adj Close  Volume
Date                                                             
2022-07-05  25.620001  25.700001  25.58  25.68      25.68  237000
00715L
2022-07-06
2022-07-07


/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWa

[*********************100%***********************]  1 of 1 completed
                 Open       High        Low  Close  Adj Close    Volume
Date                                                                   
2022-07-05  24.379999  24.690001  24.129999  24.26      24.26  11715475


/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
from apscheduler.schedulers.blocking import BlockingScheduler

from datetime import datetime


In [ ]:
sched = BlockingScheduler()

sched.add_job(daily_update, 'cron', hour='15', minute='00')

sched.start()

/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/apscheduler/util.py:436: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return tzinfo.localize(dt)
Job "daily_update (trigger: cron[hour='15', minute='0'], next run at: 2022-07-07 15:00:00 CST)" raised an exception
Traceback (most recent call last):
  File "/home/awinlab/anaconda3/envs/fl/lib/python3.7/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/tmp/ipykernel_7990/3375705848.py", line 2, in daily_update
    today = datetime.date.today()
AttributeError: 'method_descriptor' object has no attribute 'today'
Job "daily_update (trigger: cron[hour='15', minute='0'], next run at: 2022-07-08 15:00:00 CST)" raised an exception
Traceback (most rec